<a href="https://colab.research.google.com/github/Ira-a02/DataAnalysis/blob/main/ml7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from  sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм"(инструкция приведена выше).

In [5]:
newsgroups = datasets.fetch_20newsgroups(
    subset='all',
    categories=['alt.atheism', 'sci.space']
)

texts = newsgroups.data
targets = newsgroups.target

 2. Вычислите TF-IDF-признаки для всех текстов. Обратите внима
ние, что в этом задании мы предлагаем вам вычислить TF-IDF по
 всем данным. При таком подходе получается, что признаки на обу
чающем множестве используют информацию из тестовой выборки -- но такая ситуация вполне законна, поскольку мы не использу
ем значения целевой переменной из теста. На практике нередко
 встречаются ситуации, когда признаки объектов тестовой выборки
 известны на момент обучения, и поэтому можно ими пользоваться
 при обучении алгоритма.


In [21]:
vec= TfidfVectorizer()
tfidf_matrix = vec.fit_transform(texts)
#y = vec.fit_transform(targets)

AttributeError: 'numpy.int64' object has no attribute 'lower'

3. Подберите минимальныйлучшийпараметрCизмножества [10^-5, 10^-4,... , 10^4, 10^5]
 для SVM с линейным ядром (kernel=’linear’) при помощи кроссвалидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy)

In [22]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(tfidf_matrix, targets)


print("Лучший параметр C:", gs.best_params_)
print("Лучшая точность:", gs.best_score_)

Лучший параметр C: {'C': np.float64(1.0)}
Лучшая точность: 0.9932804406678872


 4. Обучите SVM по всей выборке с лучшим параметром C, найденным на предыдущем шаге.

In [26]:
final_clf = SVC(C=gs.best_params_['C'], kernel='linear', random_state=241)
final_clf.fit(tfidf_matrix, targets)

Модель обучена на всей выборке с параметром C: 1.0


 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.

In [ ]:
weights = final_clf.coef_.flatten()
feature_names = vec.get_feature_names_out()
word_weights = [(feature_names[i], weights[i]) for i in range(len(weights))]

top_words = sorted(word_weights, key=lambda x: abs(x[1]), reverse=True)[:10]

top_words_list = [word[0].lower() for word in top_words]
top_words_list.sort()
result = ",".join(top_words_list)

with open('top_words.txt', 'w') as f:
    f.write(result)

In [30]:
coef = final_clf.coef_.toarray().flatten()
feature_names = vec.get_feature_names_out()
weights_df = pd.DataFrame({'word': feature_names, 'weight': coef})
top_words = weights_df.reindex(weights_df['weight'].abs().nlargest(10).index)
res = ", ".join(top_words['word'])

with open('top_words.txt', 'w') as f:
    f.write(res)

print(res)

space, god, atheism, atheists, moon, sky, religion, bible, keith, sci
